### Korean HateSpeech Dataset

We provide the first human-annotated Korean corpus for toxic speech detection and the large unlabeled corpus.
The data is comments from the Korean entertainment news aggregation platform.

The dataset consists of 3 parts: 1) labeled 2) unlabeled and 3) news_title.

https://github.com/kocohub/korean-hate-speech

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/Users/fofx/dss15/data/korean-hate-speech-detection/train.hate.csv')

In [95]:
title = pd.read_csv('/Users/fofx/dss15/data/korean-hate-speech-detection/train.news_title.txt', sep="\t", names=['title'], header=None)

In [105]:
title.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7896 entries, 0 to 7895
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   7893 non-null   object
dtypes: object(1)
memory usage: 61.8+ KB


In [106]:
len(title)

7896

In [107]:
len(hate_speech['title'])

7896

In [3]:
data.head()

,comments,label
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,hate
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,none
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",hate
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",none
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,hate


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7896 entries, 0 to 7895
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   comments  7896 non-null   object
 1   label     7896 non-null   object
dtypes: object(2)
memory usage: 123.5+ KB


In [5]:
data['label'].value_counts()

none         3486
offensive    2499
hate         1911
Name: label, dtype: int64

In [34]:
data['comments']

0       (현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...
1       ....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...
2       ...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...
3                      1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데
4       1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...
                              ...                        
7891                                        힘내세요~ 응원합니다!!
7892                               힘내세요~~삼가 고인의 명복을 빕니다..
7893                                힘내세용 ^^ 항상 응원합니닷 ^^ !
7894    힘내소...연기로 답해요.나도 53살 인데 이런일 저런일 다 있더라구요.인격을 믿습...
7895                                   힘들면 관뒀어야지 그게 현명한거다
Name: comments, Length: 7896, dtype: object

### 전처리 방식

- Basic
    - 가장 기초적인 전처리
    - html tag 제거
    - 숫자 제거
    - Lowercasing
    - "@%*=()/+ 와 같은 punctuation 제거
- Spell check
    - 사전 기반의 오탈자 교정
    - 줄임말 원형 복원 (e.g. I'm not happy -> I am not happy)
- Part-of-Speech
    - 형태소 분석
    - Noun, Adjective, Verb, Adverb만 학습에 사용
- Stemming
    - 형태소 분석 이후 동사 원형 복원
    - Stopwords
    - 불용어 제거

### 1. Basic Preprocessing

### Korean Sentence Splitter
- 한국어 문장 분리 라이브러리 중, 가장 성능이 좋은 tokenizer 중 하나인 kss 사용

https://github.com/likejazz/korean-sentence-splitter

```pip install kss ```

In [6]:
import kss


comment_tokenized_text = []
for comment in data.comments:

    comment = comment.strip()
    for sent in kss.split_sentences(comment):
        comment_tokenized_text.append(sent.strip())

In [7]:
comment_tokenized_text # 문장 단위로 분리된 corpus

['(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네....',
 '....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을 미처 알지못했네요ㅠ',
 '...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,',
 '그래야, 공정한 사회지...',
 '심은대로 거두거라...',
 '1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데',
 '1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각이 없노',
 '10+8 진짜 이승기랑 비교된다',
 '100년안에 남녀간 성전쟁 한번 크게 치룬 후 일부다처제, 여성의 정치참여 금지, 여성 투표권 삭제가 세계의 공통문화로 자리잡을듯. 암탉이 너무 울어댐.',
 '10년뒤 윤서인은 분명히 재평가될것임. 말하나하나가 틀린게없음',
 '10년만에 재미를 느끼는 프로였는데왜 니들때문에 폐지를해야되냐',
 '10년차방탄팬인데 우리방탄처럼 성공은못하겠지만 일단 방탄의 부하가되고싶다는거니 이름기억은해둠ㅇㅇ',
 '10살 차이가 넘을텐데 부부라고?',
 '무슨 내용인지 긍금하네..',
 '12월이나 1월이더 결혼성수기지 5월은 여름이라 비수기에속한다',
 '13살 차이면 요즘 세상에 놀랄일도 아니네요',
 '1400억 뜯기고 얼마나 억울 했을꼬 뒤질만큼 아플만 하지~~~~ㅋㅋㅋㅋ',
 '15년생인 울아들은 유모차 안타는데',
 '15살차이면 어릴때사고쳤으면 딸벌인데 ㅋㅋ 능력좋네 전현무 여자들이 전현무스타일이 좋은가봐',
 '166에 57.4 다이어트 필요없지만건강위해더뺄거야다들 더마르게봄',
 '17년도 아니고 27년이면 진짜 너무했다',
 '이것이 사랑만으로 가능할수 있는지 도저히 납득이 가질 않는다',
 '180넘어갈정도의 기럭지는 아님78에서79180넘는애들은 의심.의구심을 안하고안듬그래도 비율좋아 커보이긴함',
 '180이하 호빗 한남들은 결혼 하지마셈 ㅋ 돈없으면 연애도 하지마셈 ㅋ 니들 호

In [31]:
len(comment_tokenized_text)

11337

In [36]:
data['comments'][0], comment_tokenized_text[0]

('(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네....',
 '(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네....')

In [38]:
for i in range(10):
    a = data['comments'][i], comment_tokenized_text[i]
    print(a)

('(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네....', '(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네....')
('....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을 미처 알지못했네요ㅠ', '....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을 미처 알지못했네요ㅠ')
('...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래야, 공정한 사회지...심은대로 거두거라...', '...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,')
('1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데', '그래야, 공정한 사회지...')
('1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각이 없노', '심은대로 거두거라...')
('10+8 진짜 이승기랑 비교된다', '1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데')
('100년안에 남녀간 성전쟁 한번 크게 치룬 후 일부다처제, 여성의 정치참여 금지, 여성 투표권 삭제가 세계의 공통문화로 자리잡을듯. 암탉이 너무 울어댐.', '1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각이 없노')
('10년뒤 윤서인은 분명히 재평가될것임. 말하나하나가 틀린게없음', '10+8 진짜 이승기랑 비교된다')
('10년만에 재미를 느끼는 프로였는데왜 니들때문에 폐지를해야되냐', '100년안에 남녀간 성전쟁 한번 크게 치룬 후 일부다처제, 여성의 정치참여 금지, 여성 투표권 삭제가 세계의 공통문화로 자리잡을듯. 암탉이 너무 울어댐.')
('10년차방탄팬인데 우리방탄처럼 성공은못하겠지만 일단 방탄의 부하가되고싶다는거니 이름기억은해둠ㅇㅇ', '10년뒤 윤서인은 분명히 재평가될것임. 말하나하나가 틀린게없음')


In [48]:
# define punctuation
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'

In [49]:
# map punctuation to text or standardized format
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

In [51]:
def clean_punc(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text.strip()

In [52]:
cleaned_corpus = []
for comment in data['comments']:
    cleaned_corpus.append(clean_punc(comment, punct, punct_mapping))

In [ ]:
cleaned_corpus2 = []
for comment in comment_tokenized_text:
    cleaned_corpus2.append(clean_punc(comment, punct, punct_mapping))

In [40]:
len(cleaned_corpus)

7896

In [53]:
for i in range(0, 10):
    print(cleaned_corpus[i])

( 현재 호텔주인 심정 )  아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네 .  .  .  .
.  .  .  . 한국적인 미인의 대표적인 분 .  .  . 너무나 곱고아름다운모습 .  .  . 그모습뒤의 슬픔을 미처 알지못했네요ㅠ
.  .  . 못된 넘들 .  .  . 남의 고통을 즐겼던 넘들 .  . 이젠 마땅한 처벌을 받아야지 .  .  , 그래야 ,  공정한 사회지 .  .  . 심은대로 거두거라 .  .  .
1 , 2화 어설펐는데 3 , 4화 지나서부터는 갈수록 너무 재밌던데
1 .  사람 얼굴 손톱으로 긁은것은 인격살해이고2 .  동영상이 몰카냐 ?  메걸리안들 생각이 없노
10 + 8 진짜 이승기랑 비교된다
100년안에 남녀간 성전쟁 한번 크게 치룬 후 일부다처제 ,  여성의 정치참여 금지 ,  여성 투표권 삭제가 세계의 공통문화로 자리잡을듯 .  암탉이 너무 울어댐 .
10년뒤 윤서인은 분명히 재평가될것임 .  말하나하나가 틀린게없음
10년만에 재미를 느끼는 프로였는데왜 니들때문에 폐지를해야되냐
10년차방탄팬인데 우리방탄처럼 성공은못하겠지만 일단 방탄의 부하가되고싶다는거니 이름기억은해둠ㅇㅇ


In [30]:
len(cleaned_corpus)

11337

In [55]:
data['cleaned_corpus'] = cleaned_corpus

In [56]:
data.head()

,comments,label,cleaned_corpus
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,hate,( 현재 호텔주인 심정 ) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군...
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,none,. . . . 한국적인 미인의 대표적인 분 . . . 너무나 곱고아름다운모...
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",hate,. . . 못된 넘들 . . . 남의 고통을 즐겼던 넘들 . . 이젠 마땅...
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",none,"1 , 2화 어설펐는데 3 , 4화 지나서부터는 갈수록 너무 재밌던데"
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,hate,1 . 사람 얼굴 손톱으로 긁은것은 인격살해이고2 . 동영상이 몰카냐 ? 메걸...


In [57]:
import re

def clean_comment(comments):
    corpus = []
    for i in range(0, len(comments)):
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(comments[i])) #remove punctuation
        # review = re.sub(r'\d+','', str(comments[i]))# remove number
        review = review.lower() #lower case
        review = re.sub(r'\s+', ' ', review) #remove extra space
        review = re.sub(r'<[^>]+>','',review) #remove Html tags
        review = re.sub(r'\s+', ' ', review) #remove spaces
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        
        emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
        
        review = re.sub(emoj, '', review)
        
        corpus.append(review)
    return corpus

In [58]:
basic_preprocessed_corpus = clean_comment(cleaned_corpus)

In [59]:
data['basic_preprocessed_corpus'] = basic_preprocessed_corpus

In [60]:
# 현재 계속 생기는 unicode 이모지들을 주기 적으로 업데이트 해주는 패키지
# 위에서 걸러지지 않는 이모지들을 다시 한번 걸려주기 위해 사용
import emoji

def give_emoji_free_text(text):
    allchars = [str for str in text]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])

    return clean_text

In [61]:
unicode_emoji_free_corpus = [give_emoji_free_text(text) for text in basic_preprocessed_corpus]

In [62]:
for i in range(0, 10):
    print(unicode_emoji_free_corpus[i])

현재 호텔주인 심정 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네
한국적인 미인의 대표적인 분 너무나 곱고아름다운모습 그모습뒤의 슬픔을 미처 알지못했네요ㅠ
못된 넘들 남의 고통을 즐겼던 넘들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회지 심은대로 거두거라
1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데
1 사람 얼굴 손톱으로 긁은것은 인격살해이고2 동영상이 몰카냐 메걸리안들 생각이 없노
10 8 진짜 이승기랑 비교된다
100년안에 남녀간 성전쟁 한번 크게 치룬 후 일부다처제 여성의 정치참여 금지 여성 투표권 삭제가 세계의 공통문화로 자리잡을듯 암탉이 너무 울어댐
10년뒤 윤서인은 분명히 재평가될것임 말하나하나가 틀린게없음
10년만에 재미를 느끼는 프로였는데왜 니들때문에 폐지를해야되냐
10년차방탄팬인데 우리방탄처럼 성공은못하겠지만 일단 방탄의 부하가되고싶다는거니 이름기억은해둠ㅇㅇ


In [68]:
sum(data['unicode_emoji_free_corpus'] != data['basic_preprocessed_corpus']) # emoji 패키지 사용으로 추가 57개 코멘트 청소

57

In [66]:
sum(data['unicode_emoji_free_corpus'] == data['basic_preprocessed_corpus'])

7839

In [64]:
data.head()

,comments,label,cleaned_corpus,basic_preprocessed_corpus,unicode_emoji_free_corpus
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,hate,( 현재 호텔주인 심정 ) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군...,현재 호텔주인 심정 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네,현재 호텔주인 심정 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,none,. . . . 한국적인 미인의 대표적인 분 . . . 너무나 곱고아름다운모...,한국적인 미인의 대표적인 분 너무나 곱고아름다운모습 그모습뒤의 슬픔을 미처 알지못했네요ㅠ,한국적인 미인의 대표적인 분 너무나 곱고아름다운모습 그모습뒤의 슬픔을 미처 알지못했네요ㅠ
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",hate,. . . 못된 넘들 . . . 남의 고통을 즐겼던 넘들 . . 이젠 마땅...,못된 넘들 남의 고통을 즐겼던 넘들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...,못된 넘들 남의 고통을 즐겼던 넘들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",none,"1 , 2화 어설펐는데 3 , 4화 지나서부터는 갈수록 너무 재밌던데",1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데,1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,hate,1 . 사람 얼굴 손톱으로 긁은것은 인격살해이고2 . 동영상이 몰카냐 ? 메걸...,1 사람 얼굴 손톱으로 긁은것은 인격살해이고2 동영상이 몰카냐 메걸리안들 생각이 없노,1 사람 얼굴 손톱으로 긁은것은 인격살해이고2 동영상이 몰카냐 메걸리안들 생각이 없노


### 2. Spell check

띄어쓰기 검사
- 한국어 띄어쓰기 검사 라이브러리
    - https://github.com/haven-jeon/PyKoSpacing 

맞춤법 검사
- 한국어 맞춤법 검사 라이브러리
    - https://github.com/ssut/py-hanspell

외래어 사전
- Korean-Optimized Word Representations for Out-of-Vocabulary Problems Caused by Misspelling Using Sub-character Information
    - https://link.springer.com/chapter/10.1007/978-3-030-12385-7_3

반복되는 이모티콘이나 자소 필터링
- 학습데이터를 이용하지 않으면서 데이터에 존재하는 단어를 찾거나, 문장을 단어열로 분해, 혹은 품사 판별을 할 수 있는 비지도학습 접근법
    - https://github.com/lovit/soynlp

띄어쓰기 검사기 설치

``` pip install git+https://github.com/haven-jeon/PyKoSpacing.git ```

In [18]:
from pykospacing import spacing
spacing("아버지가방에들어가신다.")

'아버지가 방에 들어가신다.'

맞춤법 검사기 설치

``` pip install git+https://github.com/ssut/py-hanspell.git ```

In [19]:
from hanspell import spell_checker
 
sent = "대채 왜 않돼는거지? 이게 더 낳은거 아닌가?"
spelled_sent = spell_checker.check(sent)
checked_sent = spelled_sent.checked

print(checked_sent)

대체 왜 안되는 거지? 이게 더 나은 거 아닌가?


이모티콘이나 자모를 normalization을 위한 라이브러리 soynlp 설치

```pip install soynlp```

In [20]:
from soynlp.normalizer import * # 중복 제거
print(repeat_normalize('믿고 있었다구우 쥐엔자아아아아아앙 ㅋㅋㅋㅋㅋㅋ ', num_repeats=2))

믿고 있었다구우 쥐엔자아아앙 ㅋㅋ


In [21]:
# 외래어 사전 다운로드
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1RNYpLE-xbMCGtiEHIoNsCmfcyJP3kLYn" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1RNYpLE-xbMCGtiEHIoNsCmfcyJP3kLYn" -o confused_loanwords.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0    667      0 --:--:-- --:--:-- --:--:--   666
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 19779  100 19779    0     0   8300      0  0:00:02  0:00:02 --:--:-- 1530k


In [22]:
loanword_map = {}
loanword_data = open('./content/confused_loanwords.txt', 'r', encoding='utf-8')

lines = loanword_data.readlines()

for line in lines:
    line = line.strip()
    miss_spell = line.split('\t')[0]
    ori_word = line.split('\t')[1]
    loanword_map[miss_spell] = ori_word

In [25]:
def spell_check_comment(texts):
    corpus = []
    for comment in texts:
        spaced_comment = spacing(comment)
        spelled_comment = spell_checker.check(comment)
        checked_comment = spelled_comment.checked
        normalized_comment = repeat_normalize(checked_comment)
        for loanword in loanword_map:
            normalized_comment = normalized_comment.replace(loanword, loanword_map[loanword])
        corpus.append(normalized_comment)
    return corpus

In [70]:
%%time 

spell_preprocessed_corpus = spell_check_comment(unicode_emoji_free_corpus)

CPU times: user 12min 18s, sys: 56.2 s, total: 13min 15s
Wall time: 18min 47s


In [71]:
data['spell_preprocessed_corpus'] = spell_preprocessed_corpus

In [72]:
data

,comments,label,cleaned_corpus,basic_preprocessed_corpus,unicode_emoji_free_corpus,spell_preprocessed_corpus
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,hate,( 현재 호텔주인 심정 ) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군...,현재 호텔주인 심정 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네,현재 호텔주인 심정 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네,현재 호텔 주인 심정 아 18 난 마른하늘에 날벼락 맞고 호텔 망하게 생겼는데 누군...
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,none,. . . . 한국적인 미인의 대표적인 분 . . . 너무나 곱고아름다운모...,한국적인 미인의 대표적인 분 너무나 곱고아름다운모습 그모습뒤의 슬픔을 미처 알지못했네요ㅠ,한국적인 미인의 대표적인 분 너무나 곱고아름다운모습 그모습뒤의 슬픔을 미처 알지못했네요ㅠ,한국적인 미인의 대표적인 분 너무나 곱고 아름다운 모습 그 모습 뒤의 슬픔을 미처 ...
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",hate,. . . 못된 넘들 . . . 남의 고통을 즐겼던 넘들 . . 이젠 마땅...,못된 넘들 남의 고통을 즐겼던 넘들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...,못된 넘들 남의 고통을 즐겼던 넘들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...,못된 놈들 남의 고통을 즐겼던 놈들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",none,"1 , 2화 어설펐는데 3 , 4화 지나서부터는 갈수록 너무 재밌던데",1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데,1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데,1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,hate,1 . 사람 얼굴 손톱으로 긁은것은 인격살해이고2 . 동영상이 몰카냐 ? 메걸...,1 사람 얼굴 손톱으로 긁은것은 인격살해이고2 동영상이 몰카냐 메걸리안들 생각이 없노,1 사람 얼굴 손톱으로 긁은것은 인격살해이고2 동영상이 몰카냐 메걸리안들 생각이 없노,1 사람 얼굴 손톱으로 긁은 것은 인격 살해이고 2 동영상이 몰카냐 에 걸리 안 들...
...,...,...,...,...,...,...
7891,힘내세요~ 응원합니다!!,none,힘내세요 ~ 응원합니다 ! !,힘내세요 응원합니다,힘내세요 응원합니다,힘내세요 응원합니다
7892,힘내세요~~삼가 고인의 명복을 빕니다..,none,힘내세요 ~ ~ 삼가 고인의 명복을 빕니다 . .,힘내세요 삼가 고인의 명복을 빕니다,힘내세요 삼가 고인의 명복을 빕니다,힘내세요 삼가 고인의 명복을 빕니다
7893,힘내세용 ^^ 항상 응원합니닷 ^^ !,none,힘내세용 ^ ^ 항상 응원합니닷 ^ ^ !,힘내세용 ^ ^ 항상 응원합니닷 ^ ^,힘내세용 ^ ^ 항상 응원합니닷 ^ ^,힘내세요 ^ ^ 항상 응원합니다 ^ ^
7894,힘내소...연기로 답해요.나도 53살 인데 이런일 저런일 다 있더라구요.인격을 믿습...,none,힘내소 . . . 연기로 답해요 . 나도 53살 인데 이런일 저런일 다 있더라구...,힘내소 연기로 답해요 나도 53살 인데 이런일 저런일 다 있더라구요 인격을 믿습니다 홨팅,힘내소 연기로 답해요 나도 53살 인데 이런일 저런일 다 있더라구요 인격을 믿습니다 홨팅,힘내소 연기로 답해요 나도 53살인데 이런 일 저런 일 다 있더라고요 인격을 믿습니...


### 3. Part-of-Speech
### Python 기반의 형태소 분석기 중, 성능이 가장 좋은 것 중 하나인 카카오의 Khaiii 설치 (Kakao Hangul Analyzer III)

khaiii는 "Kakao Hangul Analyzer III"의 첫 글자들만 모아 만든 이름으로 카카오에서 개발한 세 번째 형태소분석기입니다. 두 번째 버전의 형태소분석기 이름인 dha2 (Daumkakao Hangul Analyzer 2)를 계승한 이름이기도 합니다.

형태소는 언어학에서 일정한 의미가 있는 가장 작은 말의 단위로 발화체 내에서 따로 떼어낼 수 있는 것을 말합니다. 즉, 더 분석하면 뜻이 없어지는 말의 단위입니다. 형태소분석기는 단어를 보고 형태소 단위로 분리해내는 소프트웨어를 말합니다. 이러한 형태소분석은 자연어 처리의 가장 기초적인 절차로 이후 구문 분석이나 의미 분석으로 나아가기 위해 가장 먼저 이루어져야 하는 과정으로 볼 수 있습니다. (한국어 위키피디아에서 인용)

```!git clone https://github.com/kakao/khaiii.git
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource
!cd /content/build && make install
!cd /content/build && make package_python
!pip install /content/build/package_python``` 

In [73]:
from khaiii import KhaiiiApi
api = KhaiiiApi()

test_sents = ["너무 시시해서 죽고 싶어졌다", "가슴울리는장면들이 뇌리에 스치면서 가슴이 웅장해진다.", "싸움수준 ㄹㅇ실화냐? 진짜 세계관최강자들의 싸움이다", "정 참판 양반댁 규수 큰 교자 타고 혼례 치른 날"]
#마지막 문장은 한국어 Pangram

for sent in test_sents:
    for word in api.analyze(sent):
        for morph in word.morphs:
            print(morph.lex + '/' + morph.tag)
    print('\n')

너무/MAG
시시/XR
하/XSA
여서/EC
죽/VV
고/EC
싶/VX
어/EC
지/VX
었/EP
다/EC


가슴/NNG
울리/VV
는/ETM
장면/NNG
들/XSN
이/JKS
뇌리/NNG
에/JKB
스치/VV
면서/EC
가슴/NNG
이/JKS
웅장/XR
하/XSA
여/EC
지/VX
ㄴ다/EF
./SF


싸움수준/NNG
ㄹㅇ실화/NNG
이/VCP
냐/EF
?/SF
진짜/NNG
세계관최강자/NNG
들/XSN
의/JKG
싸움/NNG
이/VCP
다/EC


정/NNP
참판/NNG
양반/NNG
댁/XSN
규수/NNG
크/VA
ㄴ/ETM
교자/NNG
타/VV
고/EC
혼례/NNG
치르/VV
ㄴ/ETM
날/NNG




### 한글 형태소 품사 (Part Of Speech, POS) 태그표

http://kkma.snu.ac.kr/documents/?doc=postag

- NNG : 일반 명사
- NNP : 고유 명사
- NNB : 의존 명사
- VV : 동사
- VA : 형용사
- VX : 보조 용언
- MAG : 일반 부사
- MAJ : 접속 부사
- XSV : 동사 파생 접미사
- XSA : 형용사 파생 접미사

In [74]:
significant_tags = ['NNG', 'NNP', 'NNB', 'VV', 'VA', 'VX', 'MAG', 'MAJ', 'XSV', 'XSA']

def pos_text(texts):
    corpus = []
    for sent in texts:
        pos_tagged = ''
        for word in api.analyze(sent):
            for morph in word.morphs:
                if morph.tag in significant_tags:
                    pos_tagged += morph.lex + '/' + morph.tag + ' '
        corpus.append(pos_tagged.strip())
    return corpus

In [75]:
spell_preprocessed_corpus.count('') # 전처리 과정에서 이모티콘을 지우자 빈 코멘트가 생겨서 형태소 분석에 오류 생김

0

In [76]:
spell_preprocessed_corpus_copy = spell_preprocessed_corpus

In [77]:
for i in range(0, len(spell_preprocessed_corpus_copy)):
    if spell_preprocessed_corpus_copy[i] == '':
        spell_preprocessed_corpus_copy[i] = spell_preprocessed_corpus_copy[i].replace('','노 코멘트')

In [78]:
data['spell_preprocessed_corpus_copy'] = spell_preprocessed_corpus_copy

In [79]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7896 entries, 0 to 7895
Data columns (total 7 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   comments                        7896 non-null   object
 1   label                           7896 non-null   object
 2   cleaned_corpus                  7896 non-null   object
 3   basic_preprocessed_corpus       7896 non-null   object
 4   unicode_emoji_free_corpus       7896 non-null   object
 5   spell_preprocessed_corpus       7896 non-null   object
 6   spell_preprocessed_corpus_copy  7896 non-null   object
dtypes: object(7)
memory usage: 431.9+ KB


In [80]:
pos_tagged_corpus = pos_text(spell_preprocessed_corpus_copy)

In [81]:
pos_tagged_corpus

['현재/MAG 호텔/NNG 주인/NNG 심정/NNG 나/VA 마른하늘/NNG 벼락/MAG 맞/VV 호텔/NNG 망하/VV 생기/VV 계속/MAG 추모/NNG 받/VV',
 '한국/NNP 미인/NNG 대표/NNG 분/NNB 너무나/MAG 곱/VA 아름답/VA 모습/NNG 모습/NNG 뒤/NNG 슬픔/NNG 미처/MAG 알/VV 못하/VX ㅠ/NNG',
 '못되/VA 놈/NNB 남/NNG 고통/NNG 즐기/VV 놈/NNB 이제/NNG 하/XSA 처벌/NNG 받/VV 그/VV 렇/VA 공정/NNG 하/XSA 사회자/NNG 심/VV 대로/NNB 거두/VV',
 '화/NNB 어설프/VA 지나/VV 갈/MAG 록/MAG 너무/MAG 재밌/VA',
 '사람/NNG 얼굴/NNG 손톱/NNG 긁/VV 것/NNB 인격/NNG 살해/NNG 동영/NNG 상/NNP 몰카/NNG 걸/VV 리/NNP 안/MAG 들/VV 생각/NNG 없/VA',
 '진짜/NNG 이승기/NNP 비교/NNG 되/XSV',
 '년/NNB 안/NNG 남녀/NNG 간/NNB 성전쟁/NNG 한/NNG 번/NNB 크/VA 치르/VV 후/NNG 일부다처/NNG 여성/NNG 정치/NNG 참여/NNG 금지/NNG 여성/NNG 투표/NNG 삭제/NNG 세계/NNG 공통/NNG 문화/NNG 자리/NNG 잡/VV 듯/NNB 암탉/NNG 너무/MAG 울/VV 댐/VV',
 '년/NNB 뒤/NNG 윤서인/NNP 분명히/MAG 평가/NNG 되/XSV 것/NNB 말/NNG 틀리/VV 것/NNB 없/VA',
 '년/NNB 만/NNB 재미/NNG 느끼/VV 프로/NNG 왜/MAG 때문/NNB 폐지/NNG 하/VV 되/VV',
 '년/NNB 차/NNG 방탄/NNG 팬/NNG 방탄/NNG 성공/NNG 못/MAG 하/XSV 일단/MAG 방탄/NNG 부하/NNG 되/VV 싶/VX 거/NNB 이름/NNG 기억/NNG 해둠/NNG',
 '살/NNB 차이/NNG 넘/VV 터/NNB 부부/NNG 내용/NNG 하/XSA',
 '월/NNB 월/NNB 더

In [82]:
data['pos_tagged_corpus'] = pos_tagged_corpus

### 4. Stemming

동사를 원형으로 복원하기

1. NNG|NNP|NNB + XSV|XSA --> NNG|NNP|NNB + XSV|XSA + 다
2. NNG|NNP|NNB + XSA + VX --> NNG|NNP + XSA + 다
3. VV --> VV + 다
4. VX --> VX + 다

In [83]:
p1 = re.compile('[가-힣A-Za-z0-9]+/NN. [가-힣A-Za-z0-9]+/XS.')
p2 = re.compile('[가-힣A-Za-z0-9]+/NN. [가-힣A-Za-z0-9]+/XSA [가-힣A-Za-z0-9]+/VX')
p3 = re.compile('[가-힣A-Za-z0-9]+/VV')
p4 = re.compile('[가-힣A-Za-z0-9]+/VX')

In [84]:
def stemming_text(text):
    corpus = []
    for sent in text:
        ori_sent = sent
        mached_terms = re.findall(p1, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            modi_terms += '다/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)
        
        mached_terms = re.findall(p2, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                if tag != 'VX':
                    modi_terms += lemma
            modi_terms += '다/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)

        mached_terms = re.findall(p3, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            if '다' != modi_terms[-1]:
                modi_terms += '다'
            modi_terms += '/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)

        mached_terms = re.findall(p4, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            if '다' != modi_terms[-1]:
                modi_terms += '다'
            modi_terms += '/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)
        corpus.append(ori_sent)
    return corpus

In [85]:
stemming_corpus = stemming_text(pos_tagged_corpus)

In [86]:
for i in range(0, 30):
    print(stemming_corpus[i])

현재/MAG 호텔/NNG 주인/NNG 심정/NNG 나/VA 마른하늘/NNG 벼락/MAG 맞다/VV 호텔/NNG 망하다/VV 생기다/VV 계속/MAG 추모/NNG 받다/VV
한국/NNP 미인/NNG 대표/NNG 분/NNB 너무나/MAG 곱/VA 아름답/VA 모습/NNG 모습/NNG 뒤/NNG 슬픔/NNG 미처/MAG 알다/VV 못하다/VV ㅠ/NNG
못되/VA 놈/NNB 남/NNG 고통/NNG 즐기다/VV 놈/NNB 이제하다/VV 처벌/NNG 받다/VV 그다/VV 렇/VA 공정하다/VV 사회자/NNG 심다/VV 대로/NNB 거두다/VV
화/NNB 어설프/VA 지나다/VV 갈/MAG 록/MAG 너무/MAG 재밌/VA
사람/NNG 얼굴/NNG 손톱/NNG 긁다/VV 것/NNB 인격/NNG 살해/NNG 동영/NNG 상/NNP 몰카/NNG 걸다/VV 리/NNP 안/MAG 들다/VV 생각/NNG 없/VA
진짜/NNG 이승기/NNP 비교되다/VV
년/NNB 안/NNG 남녀/NNG 간/NNB 성전쟁/NNG 한/NNG 번/NNB 크/VA 치르다/VV 후/NNG 일부다처/NNG 여성/NNG 정치/NNG 참여/NNG 금지/NNG 여성/NNG 투표/NNG 삭제/NNG 세계/NNG 공통/NNG 문화/NNG 자리/NNG 잡다/VV 듯/NNB 암탉/NNG 너무/MAG 울다/VV 댐다/VV
년/NNB 뒤/NNG 윤서인/NNP 분명히/MAG 평가되다/VV 것/NNB 말/NNG 틀리다/VV 것/NNB 없/VA
년/NNB 만/NNB 재미/NNG 느끼다/VV 프로/NNG 왜/MAG 때문/NNB 폐지/NNG 하다/VV 되다/VV
년/NNB 차/NNG 방탄/NNG 팬/NNG 방탄/NNG 성공/NNG 못/MAG 하/XSV 일단/MAG 방탄/NNG 부하/NNG 되다/VV 싶다/VV 거/NNB 이름/NNG 기억/NNG 해둠/NNG
살/NNB 차이/NNG 넘다/VV 터/NNB 부부/NNG 내용하다/VV
월/NNB 월/NNB 더/MAG 결혼/NNG 성수/NNG 기지/NNG 월/NNB 여름/NNG 수기/NNG

In [87]:
data['stemming_corpus'] = stemming_corpus

### 5. Stopwords

In [89]:
stopwords = ['데/NNB', '좀/MAG', '수/NNB', '등/NNB'] # 나중에 EDA통해서 추가해야 함

In [90]:
def remove_stopword_text(text):
    corpus = []
    for sent in text:
        modi_sent = []
        for word in sent.split(' '):
            if word not in stopwords:
                modi_sent.append(word)
        corpus.append(' '.join(modi_sent))
    return corpus

In [91]:
removed_stopword_corpus = remove_stopword_text(stemming_corpus)

In [92]:
for i in range(0, 30):
    print(removed_stopword_corpus[i])

현재/MAG 호텔/NNG 주인/NNG 심정/NNG 나/VA 마른하늘/NNG 벼락/MAG 맞다/VV 호텔/NNG 망하다/VV 생기다/VV 계속/MAG 추모/NNG 받다/VV
한국/NNP 미인/NNG 대표/NNG 분/NNB 너무나/MAG 곱/VA 아름답/VA 모습/NNG 모습/NNG 뒤/NNG 슬픔/NNG 미처/MAG 알다/VV 못하다/VV ㅠ/NNG
못되/VA 놈/NNB 남/NNG 고통/NNG 즐기다/VV 놈/NNB 이제하다/VV 처벌/NNG 받다/VV 그다/VV 렇/VA 공정하다/VV 사회자/NNG 심다/VV 대로/NNB 거두다/VV
화/NNB 어설프/VA 지나다/VV 갈/MAG 록/MAG 너무/MAG 재밌/VA
사람/NNG 얼굴/NNG 손톱/NNG 긁다/VV 것/NNB 인격/NNG 살해/NNG 동영/NNG 상/NNP 몰카/NNG 걸다/VV 리/NNP 안/MAG 들다/VV 생각/NNG 없/VA
진짜/NNG 이승기/NNP 비교되다/VV
년/NNB 안/NNG 남녀/NNG 간/NNB 성전쟁/NNG 한/NNG 번/NNB 크/VA 치르다/VV 후/NNG 일부다처/NNG 여성/NNG 정치/NNG 참여/NNG 금지/NNG 여성/NNG 투표/NNG 삭제/NNG 세계/NNG 공통/NNG 문화/NNG 자리/NNG 잡다/VV 듯/NNB 암탉/NNG 너무/MAG 울다/VV 댐다/VV
년/NNB 뒤/NNG 윤서인/NNP 분명히/MAG 평가되다/VV 것/NNB 말/NNG 틀리다/VV 것/NNB 없/VA
년/NNB 만/NNB 재미/NNG 느끼다/VV 프로/NNG 왜/MAG 때문/NNB 폐지/NNG 하다/VV 되다/VV
년/NNB 차/NNG 방탄/NNG 팬/NNG 방탄/NNG 성공/NNG 못/MAG 하/XSV 일단/MAG 방탄/NNG 부하/NNG 되다/VV 싶다/VV 거/NNB 이름/NNG 기억/NNG 해둠/NNG
살/NNB 차이/NNG 넘다/VV 터/NNB 부부/NNG 내용하다/VV
월/NNB 월/NNB 더/MAG 결혼/NNG 성수/NNG 기지/NNG 월/NNB 여름/NNG 수기/NNG

In [93]:
data['removed_stopword_corpus'] = removed_stopword_corpus

In [94]:
data.head()

,comments,label,cleaned_corpus,basic_preprocessed_corpus,unicode_emoji_free_corpus,spell_preprocessed_corpus,spell_preprocessed_corpus_copy,pos_tagged_corpus,stemming_corpus,removed_stopword_corpus
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,hate,( 현재 호텔주인 심정 ) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군...,현재 호텔주인 심정 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네,현재 호텔주인 심정 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네,현재 호텔 주인 심정 아 18 난 마른하늘에 날벼락 맞고 호텔 망하게 생겼는데 누군...,현재 호텔 주인 심정 아 18 난 마른하늘에 날벼락 맞고 호텔 망하게 생겼는데 누군...,현재/MAG 호텔/NNG 주인/NNG 심정/NNG 나/VA 마른하늘/NNG 벼락/M...,현재/MAG 호텔/NNG 주인/NNG 심정/NNG 나/VA 마른하늘/NNG 벼락/M...,현재/MAG 호텔/NNG 주인/NNG 심정/NNG 나/VA 마른하늘/NNG 벼락/M...
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,none,. . . . 한국적인 미인의 대표적인 분 . . . 너무나 곱고아름다운모...,한국적인 미인의 대표적인 분 너무나 곱고아름다운모습 그모습뒤의 슬픔을 미처 알지못했네요ㅠ,한국적인 미인의 대표적인 분 너무나 곱고아름다운모습 그모습뒤의 슬픔을 미처 알지못했네요ㅠ,한국적인 미인의 대표적인 분 너무나 곱고 아름다운 모습 그 모습 뒤의 슬픔을 미처 ...,한국적인 미인의 대표적인 분 너무나 곱고 아름다운 모습 그 모습 뒤의 슬픔을 미처 ...,한국/NNP 미인/NNG 대표/NNG 분/NNB 너무나/MAG 곱/VA 아름답/VA...,한국/NNP 미인/NNG 대표/NNG 분/NNB 너무나/MAG 곱/VA 아름답/VA...,한국/NNP 미인/NNG 대표/NNG 분/NNB 너무나/MAG 곱/VA 아름답/VA...
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",hate,. . . 못된 넘들 . . . 남의 고통을 즐겼던 넘들 . . 이젠 마땅...,못된 넘들 남의 고통을 즐겼던 넘들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...,못된 넘들 남의 고통을 즐겼던 넘들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...,못된 놈들 남의 고통을 즐겼던 놈들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...,못된 놈들 남의 고통을 즐겼던 놈들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...,못되/VA 놈/NNB 남/NNG 고통/NNG 즐기/VV 놈/NNB 이제/NNG 하/...,못되/VA 놈/NNB 남/NNG 고통/NNG 즐기다/VV 놈/NNB 이제하다/VV ...,못되/VA 놈/NNB 남/NNG 고통/NNG 즐기다/VV 놈/NNB 이제하다/VV ...
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",none,"1 , 2화 어설펐는데 3 , 4화 지나서부터는 갈수록 너무 재밌던데",1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데,1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데,1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데,1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데,화/NNB 어설프/VA 지나/VV 갈/MAG 록/MAG 너무/MAG 재밌/VA,화/NNB 어설프/VA 지나다/VV 갈/MAG 록/MAG 너무/MAG 재밌/VA,화/NNB 어설프/VA 지나다/VV 갈/MAG 록/MAG 너무/MAG 재밌/VA
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,hate,1 . 사람 얼굴 손톱으로 긁은것은 인격살해이고2 . 동영상이 몰카냐 ? 메걸...,1 사람 얼굴 손톱으로 긁은것은 인격살해이고2 동영상이 몰카냐 메걸리안들 생각이 없노,1 사람 얼굴 손톱으로 긁은것은 인격살해이고2 동영상이 몰카냐 메걸리안들 생각이 없노,1 사람 얼굴 손톱으로 긁은 것은 인격 살해이고 2 동영상이 몰카냐 에 걸리 안 들...,1 사람 얼굴 손톱으로 긁은 것은 인격 살해이고 2 동영상이 몰카냐 에 걸리 안 들...,사람/NNG 얼굴/NNG 손톱/NNG 긁/VV 것/NNB 인격/NNG 살해/NNG ...,사람/NNG 얼굴/NNG 손톱/NNG 긁다/VV 것/NNB 인격/NNG 살해/NNG...,사람/NNG 얼굴/NNG 손톱/NNG 긁다/VV 것/NNB 인격/NNG 살해/NNG...


In [96]:
data.columns

Index(['comments', 'label', 'cleaned_corpus', 'basic_preprocessed_corpus',
       'unicode_emoji_free_corpus', 'spell_preprocessed_corpus',
       'spell_preprocessed_corpus_copy', 'pos_tagged_corpus',
       'stemming_corpus', 'removed_stopword_corpus'],
      dtype='object')

In [97]:
comments = data[['comments', 'cleaned_corpus', 'basic_preprocessed_corpus',
       'unicode_emoji_free_corpus', 'spell_preprocessed_corpus',
       'spell_preprocessed_corpus_copy', 'pos_tagged_corpus',
       'stemming_corpus', 'removed_stopword_corpus', 'label']]

In [98]:
comments.head()

,comments,cleaned_corpus,basic_preprocessed_corpus,unicode_emoji_free_corpus,spell_preprocessed_corpus,spell_preprocessed_corpus_copy,pos_tagged_corpus,stemming_corpus,removed_stopword_corpus,label
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,( 현재 호텔주인 심정 ) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군...,현재 호텔주인 심정 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네,현재 호텔주인 심정 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네,현재 호텔 주인 심정 아 18 난 마른하늘에 날벼락 맞고 호텔 망하게 생겼는데 누군...,현재 호텔 주인 심정 아 18 난 마른하늘에 날벼락 맞고 호텔 망하게 생겼는데 누군...,현재/MAG 호텔/NNG 주인/NNG 심정/NNG 나/VA 마른하늘/NNG 벼락/M...,현재/MAG 호텔/NNG 주인/NNG 심정/NNG 나/VA 마른하늘/NNG 벼락/M...,현재/MAG 호텔/NNG 주인/NNG 심정/NNG 나/VA 마른하늘/NNG 벼락/M...,hate
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,. . . . 한국적인 미인의 대표적인 분 . . . 너무나 곱고아름다운모...,한국적인 미인의 대표적인 분 너무나 곱고아름다운모습 그모습뒤의 슬픔을 미처 알지못했네요ㅠ,한국적인 미인의 대표적인 분 너무나 곱고아름다운모습 그모습뒤의 슬픔을 미처 알지못했네요ㅠ,한국적인 미인의 대표적인 분 너무나 곱고 아름다운 모습 그 모습 뒤의 슬픔을 미처 ...,한국적인 미인의 대표적인 분 너무나 곱고 아름다운 모습 그 모습 뒤의 슬픔을 미처 ...,한국/NNP 미인/NNG 대표/NNG 분/NNB 너무나/MAG 곱/VA 아름답/VA...,한국/NNP 미인/NNG 대표/NNG 분/NNB 너무나/MAG 곱/VA 아름답/VA...,한국/NNP 미인/NNG 대표/NNG 분/NNB 너무나/MAG 곱/VA 아름답/VA...,none
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",. . . 못된 넘들 . . . 남의 고통을 즐겼던 넘들 . . 이젠 마땅...,못된 넘들 남의 고통을 즐겼던 넘들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...,못된 넘들 남의 고통을 즐겼던 넘들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...,못된 놈들 남의 고통을 즐겼던 놈들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...,못된 놈들 남의 고통을 즐겼던 놈들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...,못되/VA 놈/NNB 남/NNG 고통/NNG 즐기/VV 놈/NNB 이제/NNG 하/...,못되/VA 놈/NNB 남/NNG 고통/NNG 즐기다/VV 놈/NNB 이제하다/VV ...,못되/VA 놈/NNB 남/NNG 고통/NNG 즐기다/VV 놈/NNB 이제하다/VV ...,hate
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데","1 , 2화 어설펐는데 3 , 4화 지나서부터는 갈수록 너무 재밌던데",1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데,1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데,1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데,1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데,화/NNB 어설프/VA 지나/VV 갈/MAG 록/MAG 너무/MAG 재밌/VA,화/NNB 어설프/VA 지나다/VV 갈/MAG 록/MAG 너무/MAG 재밌/VA,화/NNB 어설프/VA 지나다/VV 갈/MAG 록/MAG 너무/MAG 재밌/VA,none
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,1 . 사람 얼굴 손톱으로 긁은것은 인격살해이고2 . 동영상이 몰카냐 ? 메걸...,1 사람 얼굴 손톱으로 긁은것은 인격살해이고2 동영상이 몰카냐 메걸리안들 생각이 없노,1 사람 얼굴 손톱으로 긁은것은 인격살해이고2 동영상이 몰카냐 메걸리안들 생각이 없노,1 사람 얼굴 손톱으로 긁은 것은 인격 살해이고 2 동영상이 몰카냐 에 걸리 안 들...,1 사람 얼굴 손톱으로 긁은 것은 인격 살해이고 2 동영상이 몰카냐 에 걸리 안 들...,사람/NNG 얼굴/NNG 손톱/NNG 긁/VV 것/NNB 인격/NNG 살해/NNG ...,사람/NNG 얼굴/NNG 손톱/NNG 긁다/VV 것/NNB 인격/NNG 살해/NNG...,사람/NNG 얼굴/NNG 손톱/NNG 긁다/VV 것/NNB 인격/NNG 살해/NNG...,hate


In [99]:
hate_speech = pd.concat([title, comments], axis=1)

In [100]:
hate_speech.head()

,title,comments,cleaned_corpus,basic_preprocessed_corpus,unicode_emoji_free_corpus,spell_preprocessed_corpus,spell_preprocessed_corpus_copy,pos_tagged_corpus,stemming_corpus,removed_stopword_corpus,label
0,"밤새 조문 행렬…故 전미선, 동료들이 그리워하는 따뜻한 배우 [종합]",(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,( 현재 호텔주인 심정 ) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군...,현재 호텔주인 심정 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네,현재 호텔주인 심정 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네,현재 호텔 주인 심정 아 18 난 마른하늘에 날벼락 맞고 호텔 망하게 생겼는데 누군...,현재 호텔 주인 심정 아 18 난 마른하늘에 날벼락 맞고 호텔 망하게 생겼는데 누군...,현재/MAG 호텔/NNG 주인/NNG 심정/NNG 나/VA 마른하늘/NNG 벼락/M...,현재/MAG 호텔/NNG 주인/NNG 심정/NNG 나/VA 마른하늘/NNG 벼락/M...,현재/MAG 호텔/NNG 주인/NNG 심정/NNG 나/VA 마른하늘/NNG 벼락/M...,hate
1,"'연중' 故 전미선, 생전 마지막 미공개 인터뷰…환하게 웃는 모습 '먹먹'[종합]",....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,. . . . 한국적인 미인의 대표적인 분 . . . 너무나 곱고아름다운모...,한국적인 미인의 대표적인 분 너무나 곱고아름다운모습 그모습뒤의 슬픔을 미처 알지못했네요ㅠ,한국적인 미인의 대표적인 분 너무나 곱고아름다운모습 그모습뒤의 슬픔을 미처 알지못했네요ㅠ,한국적인 미인의 대표적인 분 너무나 곱고 아름다운 모습 그 모습 뒤의 슬픔을 미처 ...,한국적인 미인의 대표적인 분 너무나 곱고 아름다운 모습 그 모습 뒤의 슬픔을 미처 ...,한국/NNP 미인/NNG 대표/NNG 분/NNB 너무나/MAG 곱/VA 아름답/VA...,한국/NNP 미인/NNG 대표/NNG 분/NNB 너무나/MAG 곱/VA 아름답/VA...,한국/NNP 미인/NNG 대표/NNG 분/NNB 너무나/MAG 곱/VA 아름답/VA...,none
2,"[단독] 잔나비, 라디오 출연 취소→'한밤' 방송 연기..비판 여론 ing(종합)","...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",. . . 못된 넘들 . . . 남의 고통을 즐겼던 넘들 . . 이젠 마땅...,못된 넘들 남의 고통을 즐겼던 넘들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...,못된 넘들 남의 고통을 즐겼던 넘들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...,못된 놈들 남의 고통을 즐겼던 놈들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...,못된 놈들 남의 고통을 즐겼던 놈들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...,못되/VA 놈/NNB 남/NNG 고통/NNG 즐기/VV 놈/NNB 이제/NNG 하/...,못되/VA 놈/NNB 남/NNG 고통/NNG 즐기다/VV 놈/NNB 이제하다/VV ...,못되/VA 놈/NNB 남/NNG 고통/NNG 즐기다/VV 놈/NNB 이제하다/VV ...,hate
3,"'아스달 연대기' 장동건-김옥빈, 들끓는 '욕망커플'→눈물범벅 '칼끝 대립'","1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데","1 , 2화 어설펐는데 3 , 4화 지나서부터는 갈수록 너무 재밌던데",1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데,1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데,1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데,1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데,화/NNB 어설프/VA 지나/VV 갈/MAG 록/MAG 너무/MAG 재밌/VA,화/NNB 어설프/VA 지나다/VV 갈/MAG 록/MAG 너무/MAG 재밌/VA,화/NNB 어설프/VA 지나다/VV 갈/MAG 록/MAG 너무/MAG 재밌/VA,none
4,[DA:이슈] ‘구하라 비보’ 최종범 항소심에 영향?…법조계 “‘공소권 없음’ 아냐”,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,1 . 사람 얼굴 손톱으로 긁은것은 인격살해이고2 . 동영상이 몰카냐 ? 메걸...,1 사람 얼굴 손톱으로 긁은것은 인격살해이고2 동영상이 몰카냐 메걸리안들 생각이 없노,1 사람 얼굴 손톱으로 긁은것은 인격살해이고2 동영상이 몰카냐 메걸리안들 생각이 없노,1 사람 얼굴 손톱으로 긁은 것은 인격 살해이고 2 동영상이 몰카냐 에 걸리 안 들...,1 사람 얼굴 손톱으로 긁은 것은 인격 살해이고 2 동영상이 몰카냐 에 걸리 안 들...,사람/NNG 얼굴/NNG 손톱/NNG 긁/VV 것/NNB 인격/NNG 살해/NNG ...,사람/NNG 얼굴/NNG 손톱/NNG 긁다/VV 것/NNB 인격/NNG 살해/NNG...,사람/NNG 얼굴/NNG 손톱/NNG 긁다/VV 것/NNB 인격/NNG 살해/NNG...,hate


In [104]:
hate_speech.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7896 entries, 0 to 7895
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   title                           7893 non-null   object
 1   comments                        7896 non-null   object
 2   cleaned_corpus                  7896 non-null   object
 3   basic_preprocessed_corpus       7896 non-null   object
 4   unicode_emoji_free_corpus       7896 non-null   object
 5   spell_preprocessed_corpus       7896 non-null   object
 6   spell_preprocessed_corpus_copy  7896 non-null   object
 7   pos_tagged_corpus               7896 non-null   object
 8   stemming_corpus                 7896 non-null   object
 9   removed_stopword_corpus         7896 non-null   object
 10  label                           7896 non-null   object
dtypes: object(11)
memory usage: 678.7+ KB


In [118]:
hate_speech['comments'][1522],hate_speech['label'][1522]

("남의 일에는 신경안써' 하고 쿨한척 하던 인간들. 강남한테는 악플 쓰고 있음ㅋㅋㅋ 뭐 같은 놈들", 'hate')

In [119]:
hate_speech['comments'][5574],hate_speech['label'][5574]

('응애 응애 엄마 저 맘에 안들죠? ........아들 ?? " 너 내가 우스워 보이니? "', 'hate')

In [113]:
idx = hate_speech['title'].isnull()

In [120]:
hate_speech['comments'][7150],hate_speech['label'][7150]

("초콜릿' 드럽게 재미없다 ㅠㅠ 다들 공감하실듯", 'offensive')

In [114]:
hate_speech['title'][idx]

1522    NaN
5574    NaN
7150    NaN
Name: title, dtype: object

In [121]:
hate_speech = hate_speech.dropna()

In [122]:
hate_speech.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7893 entries, 0 to 7895
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   title                           7893 non-null   object
 1   comments                        7893 non-null   object
 2   cleaned_corpus                  7893 non-null   object
 3   basic_preprocessed_corpus       7893 non-null   object
 4   unicode_emoji_free_corpus       7893 non-null   object
 5   spell_preprocessed_corpus       7893 non-null   object
 6   spell_preprocessed_corpus_copy  7893 non-null   object
 7   pos_tagged_corpus               7893 non-null   object
 8   stemming_corpus                 7893 non-null   object
 9   removed_stopword_corpus         7893 non-null   object
 10  label                           7893 non-null   object
dtypes: object(11)
memory usage: 740.0+ KB


In [123]:
cleaned_corpus_title = []
for text in hate_speech['title']:
    cleaned_corpus_title.append(clean_punc(text, punct, punct_mapping))

In [125]:
basic_preprocessed_corpus_title = clean_comment(cleaned_corpus_title)

In [126]:
unicode_emoji_free_corpus_title = [give_emoji_free_text(text) for text in basic_preprocessed_corpus_title]

In [127]:
%%time 

spell_preprocessed_corpus_title = spell_check_comment(unicode_emoji_free_corpus_title)

CPU times: user 12min, sys: 54.4 s, total: 12min 54s
Wall time: 14min 45s


In [128]:
spell_preprocessed_corpus_title

['밤새 조문 행렬 故 전미선 동료들이 그리워하는 따뜻한 배우 [ 종합 ]',
 '연중 故 전미선 생전 마지막 미공개 인터뷰 환하게 웃는 모습 먹먹 [ 종합 ]',
 '[ 단독 ] 잔나비 라디오 출연 취소→ 한밤 방송 연기 비판 여론 ing 종합',
 '아스 달 연대기 장동건 김옥빈 들끓는 욕망커플 →눈물범벅 칼끝 대립',
 '[ da 이슈 ] 구하라 비보 최종범 항소심에 영향 법조계 공소권 없음 아냐',
 '[ 공식 입장 ] 1박2일 측 정준영 출연 중단 결정 심각성 고려',
 '배우 마크 월버그 성차별 논란에 출연료 16억 원 전액 기부',
 '[ oh 센 초점 ] 정우성 공개 저격 윤서인 이쯤 되면 정우성 안티',
 '故 설리 부재 하에 지속 어려워 악플의 밤 측이 폐지를 결정한 이유 [ 종합 ]',
 '퍼듀 2 출신 합류 누아르 롤모델 방탄소년단 종합',
 '이지훈 99억의 여자 출연 오나라와 부부 호흡 [ 공식 ]',
 '김건모 결혼 내년 5월로 연기 보다 많은 분 초청',
 '[ 종합 ] 두산家 며느리 조수애 결혼→1000만 원 부케→임신→출산 핫이슈 ing',
 '[ 룩 차이나 ] 판빙빙 급격히 살 빠져 대만 의사에 sos',
 '이병헌 이민정 4살 아들과 la 미국 여행 이병헌 디즈니랜드 섬 유모차 끌어',
 '연중 이혜성이 밝힌 전현무의 모든 것 공개 열애 반전 매력 [ 전일 야화 ]',
 '90kg 인증 몸무게 공개→실검 1위 찍은 女 연예인 [ oh 센 탐구 ]',
 '[ da 차이나 ] 이종성 2번의 이혼 끝에 27세 연하와 세 번째 결혼',
 '키 182cm 농구선수 제의 컬투쇼 권상우 바둑 빼고 다 잘하네 종합',
 '라스 남주혁 중학교 때 이미 184cm 2년 만에 키 30cm 폭풍 성장',
 '[ day 컷 ] sky 캐슬 초미녀 케이 男 아닌 女 캐릭터 위해 18kg 증량 전문',
 '할리우드 1세대 오순택 별세 편히 잠드소서 추모 물결 [ 종합 ]',
 '[ 단독 ] snl 코리아 공식적인 폐지 확정 아름다운 종료',
 '[ 단

In [130]:
hate_speech['spell_preprocessed_corpus_title'] = spell_preprocessed_corpus_title

In [131]:
hate_speech

,title,comments,cleaned_corpus,basic_preprocessed_corpus,unicode_emoji_free_corpus,spell_preprocessed_corpus,spell_preprocessed_corpus_copy,pos_tagged_corpus,stemming_corpus,removed_stopword_corpus,label,spell_preprocessed_corpus_title
0,"밤새 조문 행렬…故 전미선, 동료들이 그리워하는 따뜻한 배우 [종합]",(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,( 현재 호텔주인 심정 ) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군...,현재 호텔주인 심정 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네,현재 호텔주인 심정 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네,현재 호텔 주인 심정 아 18 난 마른하늘에 날벼락 맞고 호텔 망하게 생겼는데 누군...,현재 호텔 주인 심정 아 18 난 마른하늘에 날벼락 맞고 호텔 망하게 생겼는데 누군...,현재/MAG 호텔/NNG 주인/NNG 심정/NNG 나/VA 마른하늘/NNG 벼락/M...,현재/MAG 호텔/NNG 주인/NNG 심정/NNG 나/VA 마른하늘/NNG 벼락/M...,현재/MAG 호텔/NNG 주인/NNG 심정/NNG 나/VA 마른하늘/NNG 벼락/M...,hate,밤새 조문 행렬 故 전미선 동료들이 그리워하는 따뜻한 배우 [ 종합 ]
1,"'연중' 故 전미선, 생전 마지막 미공개 인터뷰…환하게 웃는 모습 '먹먹'[종합]",....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,. . . . 한국적인 미인의 대표적인 분 . . . 너무나 곱고아름다운모...,한국적인 미인의 대표적인 분 너무나 곱고아름다운모습 그모습뒤의 슬픔을 미처 알지못했네요ㅠ,한국적인 미인의 대표적인 분 너무나 곱고아름다운모습 그모습뒤의 슬픔을 미처 알지못했네요ㅠ,한국적인 미인의 대표적인 분 너무나 곱고 아름다운 모습 그 모습 뒤의 슬픔을 미처 ...,한국적인 미인의 대표적인 분 너무나 곱고 아름다운 모습 그 모습 뒤의 슬픔을 미처 ...,한국/NNP 미인/NNG 대표/NNG 분/NNB 너무나/MAG 곱/VA 아름답/VA...,한국/NNP 미인/NNG 대표/NNG 분/NNB 너무나/MAG 곱/VA 아름답/VA...,한국/NNP 미인/NNG 대표/NNG 분/NNB 너무나/MAG 곱/VA 아름답/VA...,none,연중 故 전미선 생전 마지막 미공개 인터뷰 환하게 웃는 모습 먹먹 [ 종합 ]
2,"[단독] 잔나비, 라디오 출연 취소→'한밤' 방송 연기..비판 여론 ing(종합)","...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",. . . 못된 넘들 . . . 남의 고통을 즐겼던 넘들 . . 이젠 마땅...,못된 넘들 남의 고통을 즐겼던 넘들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...,못된 넘들 남의 고통을 즐겼던 넘들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...,못된 놈들 남의 고통을 즐겼던 놈들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...,못된 놈들 남의 고통을 즐겼던 놈들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...,못되/VA 놈/NNB 남/NNG 고통/NNG 즐기/VV 놈/NNB 이제/NNG 하/...,못되/VA 놈/NNB 남/NNG 고통/NNG 즐기다/VV 놈/NNB 이제하다/VV ...,못되/VA 놈/NNB 남/NNG 고통/NNG 즐기다/VV 놈/NNB 이제하다/VV ...,hate,[ 단독 ] 잔나비 라디오 출연 취소→ 한밤 방송 연기 비판 여론 ing 종합
3,"'아스달 연대기' 장동건-김옥빈, 들끓는 '욕망커플'→눈물범벅 '칼끝 대립'","1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데","1 , 2화 어설펐는데 3 , 4화 지나서부터는 갈수록 너무 재밌던데",1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데,1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데,1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데,1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데,화/NNB 어설프/VA 지나/VV 갈/MAG 록/MAG 너무/MAG 재밌/VA,화/NNB 어설프/VA 지나다/VV 갈/MAG 록/MAG 너무/MAG 재밌/VA,화/NNB 어설프/VA 지나다/VV 갈/MAG 록/MAG 너무/MAG 재밌/VA,none,아스 달 연대기 장동건 김옥빈 들끓는 욕망커플 →눈물범벅 칼끝 대립
4,[DA:이슈] ‘구하라 비보’ 최종범 항소심에 영향?…법조계 “‘공소권 없음’ 아냐”,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,1 . 사람 얼굴 손톱으로 긁은것은 인격살해이고2 . 동영상이 몰카냐 ? 메걸...,1 사람 얼굴 손톱으로 긁은것은 인격살해이고2 동영상이 몰카냐 메걸리안들 생각이 없노,1 사람 얼굴 손톱으로 긁은것은 인격살해이고2 동영상이 몰카냐 메걸리안들 생각이 없노,1 사람 얼굴 손톱으로 긁은 것은 인격 살해이고 2 동영상이 몰카냐 에 걸리 안 들...,1 사람 얼굴 손톱으로 긁은 것은 인격 살해이고 2 동영상이 몰카냐 에 걸리 안 들...,사람/NNG 얼굴/NNG 손톱/NNG 긁/VV 것/NNB 인격/NNG 살해/NNG ...,사람/NNG 얼굴/NNG 손톱/NNG 긁다/VV 것/NNB 인격/NNG 살해/NNG...,사람/NNG 얼굴/NNG 손톱/NNG 긁다/VV 것/NNB 인격/NNG 살해/NNG...,hate,[ da 이슈 ] 구하라 비보 최종범 항소심에 영향 법조계 공소권 없음 아냐
...,...,...,...,...,...,...,...,...,...,...,...,...
7891,"허지웅, 허투루 넘길 말 없었다…솔직하게 드러냈던 속사정",힘내세요~ 응원합니다!!,힘내세요 ~ 응원합니다 ! !,힘내세요 응원합니다,힘내세요 응원합니다,힘내세요 응원합니다,힘내세요 응원합니다,힘/NNG 내/VV 응원/NNG 하/XSV,힘/NNG 내다/VV 응원하다/VV,힘/NNG 내다/VV 응원하다/VV,none,허지웅 허투루 넘길 말 없었다 솔직하게 드러냈던 속사정
7892,"이혜경, ‘오! 캐롤’ 공연 중 남편 오정욱 부고…오열 속 발인 [종합]",힘내세요~~삼가 고인의 명복을 빕니다..,힘내세요 ~ ~ 삼가 고인의 명복을 빕니다 . .,힘내세요 삼가 고인의 명복을 빕니다,힘내세요 삼가 고인의 명복을 빕니다,힘내세요 삼가 고인의 명복을 빕니다,힘내세요 삼가 고인의 명복을 빕니다,힘/NNG 내/VV 가/NNG 고인/NNG 명복/NNG 빌/VV,힘/NNG 내다/VV 가/NNG 고인/NNG 명복/NNG 빌다/VV,힘/NNG 내다/VV 가/NNG 고인/NNG 명복/NNG 빌다/VV,none,이혜경 오 캐럴 공연 중 남편 오정욱 부고 오열 속 발인 [ 종합 ]
7893,"'설경구♥' 송윤아, 아들과 즐거운 하루 ""전 엄마니까요""",힘내세용 ^^ 항상 응원합니닷 ^^ !,힘내세용 ^ ^ 항상 응원합니닷 ^ ^ !,힘내세용 ^ ^ 항상 응원합니닷 ^ ^,힘내세용 ^ ^ 항상 응원합니닷 ^ ^,힘내세요 ^ ^ 항상 응원합니다 ^ ^,힘내세요 ^ ^ 항상 응원합니다 ^ ^,힘/NNG 내/VV 항상/MAG 응원/NNG 하/XSV,힘/NNG 내다/VV 항상/MAG 응원하다/VV,힘/NNG 내다/VV 항상/MAG 응원하다/VV,none,설경구 송윤아 아들과 즐거운 하루 전 엄마니까요
7894,"[SC현장]""연예인 인생 협박 유감""…미소잃은 최민수, '보복운전 혐의' 2차 공판...",힘내소...연기로 답해요.나도 53살 인데 이런일 저런일 다 있더라구요.인격을 믿습...,힘내소 . . . 연기로 답해요 . 나도 53살 인데 이런일 저런일 다 있더라구...,힘내소 연기로 답해요 나도 53살 인데 이런일 저런일 다 있더라구요 인격을 믿습니다 홨팅,힘내소 연기로 답해요 나도 53살 인데 이런일 저런일 다 있더라구요 인격을 믿습니다 홨팅,힘내소

In [132]:
pos_tagged_corpus_title = pos_text(spell_preprocessed_corpus_title)

In [134]:
hate_speech['pos_tagged_corpus_title'] = pos_tagged_corpus_title

In [135]:
stemming_corpus_title = stemming_text(pos_tagged_corpus_title)

In [137]:
hate_speech['stemming_corpus_title'] = stemming_corpus_title

In [136]:
stemming_corpus_title

['밤새/NNG 조문/NNG 행렬/NNG 전미선/NNP 동료/NNG 그리워하다/VV 하/XSA 배우/NNG 종합/NNG',
 '연중/NNG 전미선/NNP 생전/NNG 마지막/NNG 공개/NNG 인터뷰/NNG 환하/VA 웃다/VV 모습/NNG 먹먹/NNG 종합/NNG',
 '단독/NNG 잔나비/NNP 라디오/NNG 출연/NNG 취소/NNG 밤/NNG 방송/NNG 연기/NNG 비판/NNG 여론/NNG 종합/NNG',
 '아스/NNG 달/NNG 연대기/NNG 장동건/NNP 김옥빈/NNP 들끓다/VV 욕망/NNG 커플/NNG 눈/MAG 물/NNG 범벅/MAG 칼끝/NNG 대립/NNG',
 '이/NNP 슈/NNG 구하/NNG 비보/NNG 최종범/NNP 항소심/NNG 영향/NNG 법조/NNG 공소/NNG 없/VA 아/NNP',
 '공식/NNG 입장/NNG 박/NNB 일/NNB 측/NNB 정준영/NNP 출연/NNG 중단/NNG 결정/NNG 고려/NNP',
 '배우/NNG 마/NNP 크/NNG 월/NNG 버/NNP 그/NNG 성/NNG 차별/NNG 논란/NNG 출연료/NNG 원/NNB 전액/NNG 기부/NNG',
 '세/VA 초점/NNG 정우성/NNP 공개/NNG 저격/NNG 윤/NNP 서/NNG 이쯤/MAG 되다/VV 정우성/NNP 안/NNP 티/NNG',
 '설/NNP 리/NNG 부재/NNG 하/NNG 지속/NNG 어렵/VA 악플/NNG 밤/NNG 측/NNG 폐지/NNG 결정하다/VV 이유/NNG 종합/NNG',
 '퍼/NNG 출신/NNG 합류/NNG 누아/NNP 르/NNG 롤모/NNG 델/NNP 방탄/NNG 소년단/NNG 종합/NNG',
 '이지훈/NNP 여자/NNG 출연/NNG 오나라/NNP 부부/NNG 호흡/NNG 공식/NNG',
 '김건모/NNP 결혼/NNG 내년/NNG 월/NNB 연기/NNG 보다/MAG 많/VA 분/NNG 초청/NNG',
 '종합/NNG 두산/NNG 며느리/NNG 조수애/NNG 결혼/NNG 원/NNB 부/NNP 하다/VV 임신/NNG 출산/NN

In [140]:
removed_stopword_corpus_title = remove_stopword_text(stemming_corpus_title)

In [141]:
removed_stopword_corpus_title

['밤새/NNG 조문/NNG 행렬/NNG 전미선/NNP 동료/NNG 그리워하다/VV 하/XSA 배우/NNG 종합/NNG',
 '연중/NNG 전미선/NNP 생전/NNG 마지막/NNG 공개/NNG 인터뷰/NNG 환하/VA 웃다/VV 모습/NNG 먹먹/NNG 종합/NNG',
 '단독/NNG 잔나비/NNP 라디오/NNG 출연/NNG 취소/NNG 밤/NNG 방송/NNG 연기/NNG 비판/NNG 여론/NNG 종합/NNG',
 '아스/NNG 달/NNG 연대기/NNG 장동건/NNP 김옥빈/NNP 들끓다/VV 욕망/NNG 커플/NNG 눈/MAG 물/NNG 범벅/MAG 칼끝/NNG 대립/NNG',
 '이/NNP 슈/NNG 구하/NNG 비보/NNG 최종범/NNP 항소심/NNG 영향/NNG 법조/NNG 공소/NNG 없/VA 아/NNP',
 '공식/NNG 입장/NNG 박/NNB 일/NNB 측/NNB 정준영/NNP 출연/NNG 중단/NNG 결정/NNG 고려/NNP',
 '배우/NNG 마/NNP 크/NNG 월/NNG 버/NNP 그/NNG 성/NNG 차별/NNG 논란/NNG 출연료/NNG 원/NNB 전액/NNG 기부/NNG',
 '세/VA 초점/NNG 정우성/NNP 공개/NNG 저격/NNG 윤/NNP 서/NNG 이쯤/MAG 되다/VV 정우성/NNP 안/NNP 티/NNG',
 '설/NNP 리/NNG 부재/NNG 하/NNG 지속/NNG 어렵/VA 악플/NNG 밤/NNG 측/NNG 폐지/NNG 결정하다/VV 이유/NNG 종합/NNG',
 '퍼/NNG 출신/NNG 합류/NNG 누아/NNP 르/NNG 롤모/NNG 델/NNP 방탄/NNG 소년단/NNG 종합/NNG',
 '이지훈/NNP 여자/NNG 출연/NNG 오나라/NNP 부부/NNG 호흡/NNG 공식/NNG',
 '김건모/NNP 결혼/NNG 내년/NNG 월/NNB 연기/NNG 보다/MAG 많/VA 분/NNG 초청/NNG',
 '종합/NNG 두산/NNG 며느리/NNG 조수애/NNG 결혼/NNG 원/NNB 부/NNP 하다/VV 임신/NNG 출산/NN

In [142]:
hate_speech['removed_stopword_corpus_title'] = removed_stopword_corpus_title

In [143]:
hate_speech.columns

Index(['title', 'comments', 'cleaned_corpus', 'basic_preprocessed_corpus',
       'unicode_emoji_free_corpus', 'spell_preprocessed_corpus',
       'spell_preprocessed_corpus_copy', 'pos_tagged_corpus',
       'stemming_corpus', 'removed_stopword_corpus', 'label',
       'spell_preprocessed_corpus_title', 'pos_tagged_corpus_title',
       'stemming_corpus_title', 'removed_stopword_corpus_title'],
      dtype='object')

In [144]:
hate_speech = hate_speech[['title', 'spell_preprocessed_corpus_title', 'pos_tagged_corpus_title',
       'stemming_corpus_title', 'removed_stopword_corpus_title', 'comments', 'cleaned_corpus', 'basic_preprocessed_corpus',
       'unicode_emoji_free_corpus', 'spell_preprocessed_corpus',
       'spell_preprocessed_corpus_copy', 'pos_tagged_corpus',
       'stemming_corpus', 'removed_stopword_corpus', 'label']]

In [145]:
hate_speech.head()

,title,spell_preprocessed_corpus_title,pos_tagged_corpus_title,stemming_corpus_title,removed_stopword_corpus_title,comments,cleaned_corpus,basic_preprocessed_corpus,unicode_emoji_free_corpus,spell_preprocessed_corpus,spell_preprocessed_corpus_copy,pos_tagged_corpus,stemming_corpus,removed_stopword_corpus,label
0,"밤새 조문 행렬…故 전미선, 동료들이 그리워하는 따뜻한 배우 [종합]",밤새 조문 행렬 故 전미선 동료들이 그리워하는 따뜻한 배우 [ 종합 ],밤새/NNG 조문/NNG 행렬/NNG 전미선/NNP 동료/NNG 그리워하/VV 하/...,밤새/NNG 조문/NNG 행렬/NNG 전미선/NNP 동료/NNG 그리워하다/VV 하...,밤새/NNG 조문/NNG 행렬/NNG 전미선/NNP 동료/NNG 그리워하다/VV 하...,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,( 현재 호텔주인 심정 ) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군...,현재 호텔주인 심정 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네,현재 호텔주인 심정 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네,현재 호텔 주인 심정 아 18 난 마른하늘에 날벼락 맞고 호텔 망하게 생겼는데 누군...,현재 호텔 주인 심정 아 18 난 마른하늘에 날벼락 맞고 호텔 망하게 생겼는데 누군...,현재/MAG 호텔/NNG 주인/NNG 심정/NNG 나/VA 마른하늘/NNG 벼락/M...,현재/MAG 호텔/NNG 주인/NNG 심정/NNG 나/VA 마른하늘/NNG 벼락/M...,현재/MAG 호텔/NNG 주인/NNG 심정/NNG 나/VA 마른하늘/NNG 벼락/M...,hate
1,"'연중' 故 전미선, 생전 마지막 미공개 인터뷰…환하게 웃는 모습 '먹먹'[종합]",연중 故 전미선 생전 마지막 미공개 인터뷰 환하게 웃는 모습 먹먹 [ 종합 ],연중/NNG 전미선/NNP 생전/NNG 마지막/NNG 공개/NNG 인터뷰/NNG 환...,연중/NNG 전미선/NNP 생전/NNG 마지막/NNG 공개/NNG 인터뷰/NNG 환...,연중/NNG 전미선/NNP 생전/NNG 마지막/NNG 공개/NNG 인터뷰/NNG 환...,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,. . . . 한국적인 미인의 대표적인 분 . . . 너무나 곱고아름다운모...,한국적인 미인의 대표적인 분 너무나 곱고아름다운모습 그모습뒤의 슬픔을 미처 알지못했네요ㅠ,한국적인 미인의 대표적인 분 너무나 곱고아름다운모습 그모습뒤의 슬픔을 미처 알지못했네요ㅠ,한국적인 미인의 대표적인 분 너무나 곱고 아름다운 모습 그 모습 뒤의 슬픔을 미처 ...,한국적인 미인의 대표적인 분 너무나 곱고 아름다운 모습 그 모습 뒤의 슬픔을 미처 ...,한국/NNP 미인/NNG 대표/NNG 분/NNB 너무나/MAG 곱/VA 아름답/VA...,한국/NNP 미인/NNG 대표/NNG 분/NNB 너무나/MAG 곱/VA 아름답/VA...,한국/NNP 미인/NNG 대표/NNG 분/NNB 너무나/MAG 곱/VA 아름답/VA...,none
2,"[단독] 잔나비, 라디오 출연 취소→'한밤' 방송 연기..비판 여론 ing(종합)",[ 단독 ] 잔나비 라디오 출연 취소→ 한밤 방송 연기 비판 여론 ing 종합,단독/NNG 잔나비/NNP 라디오/NNG 출연/NNG 취소/NNG 밤/NNG 방송/...,단독/NNG 잔나비/NNP 라디오/NNG 출연/NNG 취소/NNG 밤/NNG 방송/...,단독/NNG 잔나비/NNP 라디오/NNG 출연/NNG 취소/NNG 밤/NNG 방송/...,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",. . . 못된 넘들 . . . 남의 고통을 즐겼던 넘들 . . 이젠 마땅...,못된 넘들 남의 고통을 즐겼던 넘들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...,못된 넘들 남의 고통을 즐겼던 넘들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...,못된 놈들 남의 고통을 즐겼던 놈들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...,못된 놈들 남의 고통을 즐겼던 놈들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...,못되/VA 놈/NNB 남/NNG 고통/NNG 즐기/VV 놈/NNB 이제/NNG 하/...,못되/VA 놈/NNB 남/NNG 고통/NNG 즐기다/VV 놈/NNB 이제하다/VV ...,못되/VA 놈/NNB 남/NNG 고통/NNG 즐기다/VV 놈/NNB 이제하다/VV ...,hate
3,"'아스달 연대기' 장동건-김옥빈, 들끓는 '욕망커플'→눈물범벅 '칼끝 대립'",아스 달 연대기 장동건 김옥빈 들끓는 욕망커플 →눈물범벅 칼끝 대립,아스/NNG 달/NNG 연대기/NNG 장동건/NNP 김옥빈/NNP 들끓/VV 욕망/...,아스/NNG 달/NNG 연대기/NNG 장동건/NNP 김옥빈/NNP 들끓다/VV 욕망...,아스/NNG 달/NNG 연대기/NNG 장동건/NNP 김옥빈/NNP 들끓다/VV 욕망...,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데","1 , 2화 어설펐는데 3 , 4화 지나서부터는 갈수록 너무 재밌던데",1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데,1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데,1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데,1 2화 어설펐는데 3 4화 지나서부터는 갈수록 너무 재밌던데,화/NNB 어설프/VA 지나/VV 갈/MAG 록/MAG 너무/MAG 재밌/VA,화/NNB 어설프/VA 지나다/VV 갈/MAG 록/MAG 너무/MAG 재밌/VA,화/NNB 어설프/VA 지나다/VV 갈/MAG 록/MAG 너무/MAG 재밌/VA,none
4,[DA:이슈] ‘구하라 비보’ 최종범 항소심에 영향?…법조계 “‘공소권 없음’ 아냐”,[ da 이슈 ] 구하라 비보 최종범 항소심에 영향 법조계 공소권 없음 아냐,이/NNP 슈/NNG 구하/NNG 비보/NNG 최종범/NNP 항소심/NNG 영향/N...,이/NNP 슈/NNG 구하/NNG 비보/NNG 최종범/NNP 항소심/NNG 영향/N...,이/NNP 슈/NNG 구하/NNG 비보/NNG 최종범/NNP 항소심/NNG 영향/N...,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,1 . 사람 얼굴 손톱으로 긁은것은 인격살해이고2 . 동영상이 몰카냐 ? 메걸...,1 사람 얼굴 손톱으로 긁은것은 인격살해이고2 동영상이 몰카냐 메걸리안들 생각이 없노,1 사람 얼굴 손톱으로 긁은것은 인격살해이고2 동영상이 몰카냐 메걸리안들 생각이 없노,1 사람 얼굴 손톱으로 긁은 것은 인격 살해이고 2 동영상이 몰카냐 에 걸리 안 들...,1 사람 얼굴 손톱으로 긁은 것은 인격 살해이고 2 동영상이 몰카냐 에 걸리 안 들...,사람/NNG 얼굴/NNG 손톱/NNG 긁/VV 것/NNB 인격/NNG 살해/NNG ...,사람/NNG 얼굴/NNG 손톱/NNG 긁다/VV 것/NNB 인격/NNG 살해/NNG...,사람/NNG 얼굴/NNG 손톱/NNG 긁다/VV 것/NNB 인격/NNG 살해/NNG...,hate


In [146]:
hate_speech.to_csv('hate_speech.csv')